## 알라딘 OpenAPI 가이드 
* (최종 수정일: 2022.7.13)
* 출처 : https://blog.aladin.co.kr/openapi/6695306
* [알라딘 Open API 매뉴얼 - Google Docs](https://docs.google.com/document/d/1mX-WxuoGs8Hy-QalhHcvuV17n50uGI2Sg_GHofgiePE/edit)



## OpenAPI 개요

OpenAPI는 크게 3가지로 나뉩니다:

1. 검색 API
2. 상품 API (단일 상품 조회)
3. 상품 리스트 API (베스트셀러 등 다양한 상품 리스트)

- 상품 API는 단일 상품 조회로, 검색 API보다 더 자세한 부가 정보를 제공합니다.
- 모든 API 결과는 XML(기본값) 또는 JSON 형식으로 제공됩니다.
- GET 및 POST 방식 모두 지원됩니다.
- API 사용은 일일 5,000회로 제한됩니다. (추가 사용은 별도 문의)

## 상품 리스트 API

### 제공 리스트 종류

- 신간 전체 리스트
- 주목할 만한 신간 리스트
- 편집자 추천 리스트 (카테고리별 조회 - 국내도서/음반/외서만 지원)
- 베스트셀러
- 북플 베스트셀러 (국내도서만 조회 가능)

### 요청 방법

- 요청 URL: `http://www.aladin.co.kr/ttb/api/ItemList.aspx`
- 요청 URL 샘플:
  ```
  http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey=TTBKey&QueryType=ItemNewAll&MaxResults=10&start=1&SearchTarget=Book&output=xml&Version=20131101
  ```

### 결과 샘플

- XML 형식: [http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.xml](http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.xml)
- JavaScript 형식: [http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.js](http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.js)

**참고**: 총 결과는 최대 1,000개까지만 조회 가능합니다.

* 터미널에서 환경변수 설정하기
    * Linux/Mac: export ALADIN_TTB_KEY=your_ttb_key_here
    * Windows: set ALADIN_TTB_KEY=your_ttb_key_here

In [ ]:
import os
# os.environ['ALADIN_TTB_KEY'] = "ttb-------"
os.environ.get('ALADIN_TTB_KEY')

In [ ]:
import requests
import xml.etree.ElementTree as ET
import sqlite3
import os
import time


def search_books(query, max_results=50, start=1):
    base_url = "http://www.aladin.co.kr/ttb/api/ItemSearch.aspx"
    
    ttbkey = os.environ.get('ALADIN_TTB_KEY')
    if not ttbkey:
        raise ValueError("ALADIN_TTB_KEY 환경 변수가 설정되지 않았습니다.")
    
    params = {
        'TTBKey': ttbkey,
        'Query': query,
        'QueryType': 'Keyword',
        'MaxResults': max_results,
        'start': start,
        'SearchTarget': "Book",
        'output': 'xml',
        'Version': '20131101'
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        namespaces = {'ns': 'http://www.aladin.co.kr/ttb/apiguide.aspx'}
        root = ET.fromstring(response.content)
        items = root.findall('.//ns:item', namespaces)
        
        if not items:
            return None
        
        books = []
        for item in items:
            book = {
                'title': item.find('ns:title', namespaces).text,
                'author': item.find('ns:author', namespaces).text,
                'isbn13': item.find('ns:isbn13', namespaces).text,
                'publisher': item.find('ns:publisher', namespaces).text,
                'pubdate': item.find('ns:pubDate', namespaces).text,
                'price': int(item.find('ns:priceStandard', namespaces).text),
                'description': item.find('ns:description', namespaces).text
            }
            books.append(book)
        
        return books
    except requests.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None
    except ET.ParseError as e:
        print(f"XML 파싱 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")
        return None

def collect_all_books(query, max_results=50):
    conn = create_database()
    cursor = conn.cursor()
    total_books = 0
    db_count = 0
    
    print(f"API에서 도서 수집 시작")
    
    start = 1
    while True:
        
        print(f"total_books: {total_books}, DB 도서 수: {db_count}, start: {start}")
        
        books = search_books(query, max_results, start)
        if not books:
            break
        
        insert_books(conn, books)
        total_books += len(books)
        
        cursor.execute("SELECT COUNT(*) FROM books")
        new_db_count = cursor.fetchone()[0]
        
        if new_db_count == db_count:
            print(f"새로운 도서가 더 이상 추가되지 않습니다.")
            break
        
        db_count = new_db_count
        
        start += max_results
        
        if start > 200:
            break
        time.sleep(1)
    
    conn.close()
    print(f"'{query}' 검색 결과:")
    print(f"API에서 총 {total_books}개의 도서 정보를 수집했습니다.")
    print(f"데이터베이스에 저장된 최종 고유한 도서 수: {db_count}")

def create_database():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT,
        isbn13 TEXT UNIQUE,
        publisher TEXT,
        pubdate TEXT,
        price INTEGER,
        description TEXT
    )
    ''')
    conn.commit()
    return conn

def insert_books(conn, books):
    if not books:
        return
    
    cursor = conn.cursor()
    
    print("book 0: ", books[0]['title'])
    for book in books:
        try:
            cursor.execute('''
            INSERT OR IGNORE INTO books (title, author, isbn13, publisher, pubdate, price, description)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (book['title'], book['author'], book['isbn13'], book['publisher'], book['pubdate'], book['price'], book['description']))
        except sqlite3.IntegrityError:
            pass  # 중복된 ISBN은 무시
    conn.commit()


def verify_data():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM books")
    count = cursor.fetchone()[0]
    
    print(f"\n데이터베이스의 총 도서 수: {count}")
    
    conn.close()

# 메인 실행 부분
if __name__ == "__main__":
    try:
        query = '프로그래밍'
        collect_all_books(query)
        verify_data()
    except Exception as e:
        print(f"프로그램 실행 중 오류 발생: {e}")

In [ ]:
verify_data()

In [ ]:
len(books)

In [ ]:
result = search_books(query, max_results=100, start=1)
len(result), result[-1]["title"]

In [ ]:
import sqlite3
import pandas as pd

def get_books_dataframe():
    # SQLite 데이터베이스에 연결
    conn = sqlite3.connect('books.db')
    
    # SQL 쿼리 실행 및 데이터프레임으로 변환
    query = "SELECT * FROM books"
    df = pd.read_sql_query(query, conn)
    
    # 데이터베이스 연결 종료
    conn.close()
    
    return df

def print_dataframe_info(df):
    print("데이터프레임 정보:")
    print(f"행 수: {df.shape[0]}")
    print(f"열 수: {df.shape[1]}")
    print("\n컬럼명:")
    print(df.columns.tolist())
    print("\n데이터 타입:")
    print(df.dtypes)
    display("\n처음 5개 행:")
    print(df.head())
    display("\n기본 통계 정보:")
    display(df.describe())

# 메인 실행 부분
if __name__ == "__main__":
    # 데이터프레임 가져오기
    books_df = get_books_dataframe()
    
    # 데이터프레임 정보 출력
    print_dataframe_info(books_df)

In [ ]:
books_df